### Introduction
This a test

#### 0) Simulation parameters

In [23]:
# Simulation general parameters
DAYS_TO_EXP     = 1 # days until expiration
N_SIMS          = 200 # number of simulations
REBAL_FREQ      = 1/252 # frequency of rebalancing
INI_PRICE       = 100 # stock initial price


# Simulation technical details
DRIFT           = 0.0 # stocks drift
ANNUALIZED_VOL  = 0.2 # process volatility
NDAYS_YEAR      = 252 # number of days per year


#### 00) Libraries

In [24]:
import numpy as np
import pandas as pd
from scipy.stats import norm, lognorm

#### 000) Black-Scholes functions

In [25]:
# Define Black-Scholes functions
def calculate_call_delta(S, K, r, T, sigma):
    """
    Calculate the delta of an European call option using the Black-Scholes formula.

    Parameters:
    S (float): Current stock price.
    K (float): Strike price.
    r (float): Risk-free rate.
    T (float): Time to expiration (in years).
    sigma (float): Volatility.

    Returns:
    float: Delta of the call option.
    """
    
    d1 = (np.log(S / K) + (r + 0.5 * sigma**2) * T) / (sigma * np.sqrt(T))
    delta = norm.cdf(d1)
    return delta

#### 1) Simulate the Brownian process

Some details:
- Returns will be simulated as lognormal. Let's say that the lognormal distribution of our process has mean $\mu$ and standard deviation $\sigma$. Then, the expected value of the random variable will be $e^{\mu + \sigma^2/2}$. It is important to realize that even if there is no drift, $\mu = 0$, the expected value will be afected by the variance, $e^{\sigma^2/2}$. Therefore, to simulate stock paths with zero drift it is necessary to substract $\sigma^2/2$ from the random variable. If not, the stock prices path will be drift.
- Test2

In [40]:
# Daily volatility
daily_vol = ANNUALIZED_VOL/np.sqrt(NDAYS_YEAR)

# Create a log-normal distribution object (sigma^2/2 is substracted from the mean)
lognorm_dist = lognorm(daily_vol, 
                       loc=0.0, 
                       scale=np.exp(DRIFT - 0.5*(daily_vol)**2))

# Dataframe containing simulated paths
dfPaths = pd.DataFrame(np.nan, 
                       index = np.arange(start = 0.0, stop = DAYS_TO_EXP, step = REBAL_FREQ), 
                       columns = [f'Path{i}' for i in range(1, N_SIMS+1)])

# Simulate each path
for column in dfPaths:
    for idx, step in enumerate(dfPaths[column]):

        if idx == 0:
            dfPaths[column].iloc[0] = INI_PRICE
        else:
            dfPaths[column].iloc[idx] = dfPaths[column].iloc[idx-1]*lognorm_dist.ppf(np.random.random())
        

1
nan
2
nan
3
nan
4
nan
5
nan
6
nan
7
nan
8
nan
9
nan
10
nan
11
nan
12
nan
13
nan
14
nan
15
nan
16
nan
17
nan
18
nan
19
nan
20
nan
21
nan
22
nan
23
nan
24
nan
25
nan
26
nan
27
nan
28
nan
29
nan
30
nan
31
nan
32
nan
33
nan
34
nan
35
nan
36
nan
37
nan
38
nan
39
nan
40
nan
41
nan
42
nan
43
nan
44
nan
45
nan
46
nan
47
nan
48
nan
49
nan
50
nan
51
nan
52
nan
53
nan
54
nan
55
nan
56
nan
57
nan
58
nan
59
nan
60
nan
61
nan
62
nan
63
nan
64
nan
65
nan
66
nan
67
nan
68
nan
69
nan
70
nan
71
nan
72
nan
73
nan
74
nan
75
nan
76
nan
77
nan
78
nan
79
nan
80
nan
81
nan
82
nan
83
nan
84
nan
85
nan
86
nan
87
nan
88
nan
89
nan
90
nan
91
nan
92
nan
93
nan
94
nan
95
nan
96
nan
97
nan
98
nan
99
nan
100
nan
101
nan
102
nan
103
nan
104
nan
105
nan
106
nan
107
nan
108
nan
109
nan
110
nan
111
nan
112
nan
113
nan
114
nan
115
nan
116
nan
117
nan
118
nan
119
nan
120
nan
121
nan
122
nan
123
nan
124
nan
125
nan
126
nan
127
nan
128
nan
129
nan
130
nan
131
nan
132
nan
133
nan
134
nan
135
nan
136
nan
137
nan
138
nan
139


KeyboardInterrupt: 